In [ ]:
import numpy as np
import time
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D,Conv2D
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
from keras.layers import Dropout
from keras.layers import Bidirectional
from keras.layers import TimeDistributed, Dense, Flatten,ConvLSTM2D,RepeatVector
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import tensorflow as tf 
from keras import backend as k 


from sklearn.preprocessing import MinMaxScaler

#from tensorflow.keras import Sequential
#from tensorflow.keras.layers import Conv2D, Flatten, Dense, TimeDistributed, ConvLSTM2D, RepeatVector

In [ ]:
#df1=pd.read_csv('https://raw.githubusercontent.com/bre3760/ictbuilding/dev/data/anno1/eplusout/eplusout.csv')
#df2=pd.read_csv('https://raw.githubusercontent.com/bre3760/ictbuilding/dev/data/anno2/eplusout/eplusout.csv')
#df3=pd.read_csv('https://raw.githubusercontent.com/bre3760/ictbuilding/dev/data/anno3/eplusout/eplusout.csv')
df1=pd.read_csv('/home/ict4bd/Anno1/eplusout.csv')
df2=pd.read_csv('/home/ict4bd/Anno2/eplusout.csv')
df3=pd.read_csv('/home/ict4bd/Anno3/eplusout.csv')

In [ ]:
#Set number of timestep to use for prediction and to predict
n_in = 12
n_out = 6
verbose=False

In [ ]:
def changetime(df,year):
    datetimeform = []
    for i in range(len(df.index)):
        tt = df["Date/Time"][i]
        days, hours = tt.split('  ')
        tt = f'{days.split("/")[1]}/{days.split("/")[0]}/{year}{hours}'
        tt = tt.replace(' ', '')
        if '201724:' or '201824:' or '201924:' in tt:
            tt=tt.replace('24:', '00:')
            timestamp = time.mktime(datetime.strptime(tt, "%d/%m/%Y%H:%M:%S").timetuple())
            timestamp += 86400
            #timestamp = datetime.fromtimestamp(timestamp)

        else:
            timestamp = time.mktime(datetime.strptime(tt, "%d/%m/%Y%H:%M:%S").timetuple())
            #timestamp = datetime.fromtimestamp(timestamp)
        datetimeform.append(timestamp)

    df["TimeStep"]=datetimeform
    
changetime(df1,"2017")
changetime(df2,"2018")
changetime(df3,"2019")
df1.head()

#Extract mean temp time series
#mean value of the 3 indoor zones
def meanDF(df):
    df_mean = pd.DataFrame()
    #"Date/Time",\
    #"Environment:Site Outdoor Air Drybulb Temperature [C](TimeStep)",\
    #"Electricity:Facility [J](TimeStep)"
    df_mean = df[[
              "BLOCK1:BEDROOM:Zone Mean Air Temperature [C](TimeStep:ON)",\
              "BLOCK1:BATHROOM:Zone Mean Air Temperature [C](TimeStep:ON)",\
              "BLOCK1:KITCHEN:Zone Mean Air Temperature [C](TimeStep:ON)",\
              ]]
    df_temp=df_mean.copy()
    df_temp.dropna(inplace=True)
    df_temp["mean"] = df_temp.mean(1)


    df_final=pd.DataFrame()
    df_final["DateTime"] = df["TimeStep"]
    df_final["Outdoor_Mean"] = df["Environment:Site Outdoor Air Drybulb Temperature [C](TimeStep)"]
    df_final["Indoor_Mean"] = df_temp["mean"]
    df_final["Cooling"] = df["DistrictCooling:Facility [J](TimeStep)"]
    df_final["Power"]=df["Electricity:Facility [J](TimeStep)"]
    df_final["Delta_T"] = df_final["Indoor_Mean"]-df_final["Outdoor_Mean"]
    
    #df_final
    return df_final

df1_final=meanDF(df1)
df2_final=meanDF(df2)
df3_final=meanDF(df3)
df1_final.head()

In [ ]:
# selecting only the values to train the rnn on 
x = pd.DataFrame()
meas = [
        #"TimeStep",
        "DistrictCooling:Facility [J](TimeStep)",
        "Electricity:Facility [J](TimeStep)",
        "Environment:Site Outdoor Air Drybulb Temperature [C](TimeStep)",
        "Environment:Site Direct Solar Radiation Rate per Area [W/m2](TimeStep)",
        "Environment:Site Wind Speed [m/s](TimeStep)",
        "Environment:Site Outdoor Air Barometric Pressure [Pa](TimeStep)",
        "Environment:Site Wind Direction [deg](TimeStep)",
        "Environment:Site Diffuse Solar Radiation Rate per Area [W/m2](TimeStep)",
        "BLOCK1:BEDROOM:Zone Mean Air Temperature [C](TimeStep:ON)",
        "BLOCK1:BATHROOM:Zone Mean Air Temperature [C](TimeStep:ON)",
        "BLOCK1:KITCHEN:Zone Mean Air Temperature [C](TimeStep:ON)"]

#x["Indoor_Mean"] = pd.concat([df1_final["Indoor_Mean"],df2_final["Indoor_Mean"],df3_final["Indoor_Mean"]])

for m in meas:
    x[m]=pd.concat([df1[m],df2[m],df3[m]])
#x.set_index("TimeStep",inplace=True)
#x.head()
x.reset_index(inplace=True,drop=True)

In [ ]:
column_indices = {name: i for i, name in enumerate(x.columns)}
x.describe().transpose()

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
reframed = series_to_supervised(x.to_numpy(), n_in, n_out)
if verbose:
    reframed.head()
#reframed.shape

#range(x.shape[1]*n_in+1,x.shape[1]*n_in+x.shape[1])

In [ ]:
columns=[]
for i in range(n_out):
    for c in reframed.columns[range(x.shape[1]*n_in+1+i*x.shape[1],x.shape[1]*n_in+ (i+1)*x.shape[1])]:
        columns.append(c)
if verbose:
    print(columns)
reframed.drop(columns, axis=1, inplace=True)
reframed.head()

In [ ]:
#n_in = 6
#n_out = 2
#reframed = series_to_supervised(x.to_numpy(), n_in, n_out)
#reframed.drop(reframed.columns[range(x.shape[1]*n_in+1,x.shape[1]*n_in+x.shape[1])], axis=1, inplace=True)
#reframed.head()
#reframed.shape

In [ ]:
# Split dataset into train, validation and test and normalize data

ntot = reframed.shape[0]
values = reframed.values

#first two years for training 
x_train = values[:(len(df1)+len(df2)), :]
#x_train.set_index("TimeStep",inplace=True)
mean = x_train.mean()
std = x_train.std()
x_train_norm = (x_train - mean)/std
print("Training set dimension: ",len(x_train))

x_val =  values[(len(df1)+len(df2)): (len(df1)+len(df2)) + int(0.5*len(df3)), :]
#x_val.set_index("TimeStep",inplace=True)
x_val_norm = (x_val-mean)/std
print("Validation set dimension: ",len(x_val))

x_test =  values[(len(df1)+len(df2)) + int(0.5*len(df3)):, :]
#x_test.set_index("TimeStep",inplace=True)
x_test_norm = (x_test-mean)/std
print("Test set dimension: ",len(x_test))

train_X, train_y = x_train_norm[:, :-n_out],  x_train_norm[:, -n_out:]
val_X, val_y = x_val_norm[:, :-n_out],  x_val_norm[:, -n_out:]
test_X, test_y = x_test_norm[:, :-n_out],  x_test_norm[:, -n_out:]

In [ ]:
def bi_lstm_model():
	model = Sequential()
	model.add(TimeDistributed(Conv1D(filters=64, kernel_size=2, activation='relu'), input_shape=(None, n_steps, n_features)))
	model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
	model.add(TimeDistributed(Conv1D(filters=64, kernel_size=2,activation='relu')))
	model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
	model.add(TimeDistributed(Flatten()))
	model.add(Bidirectional(LSTM(64, activation='relu',return_sequences=True)))
	model.add(Bidirectional(LSTM(64, activation='relu')))
	model.add(Dense(128))
	model.add(Dropout(0.5))
	model.add(Dense(n_out))
	model.compile(optimizer='adam', loss='mse',metrics = ['mae','mse','mape'])
	return model

In [ ]:
#CNN-Bi-LSTM 
#n_steps, n_features = train_X.shape[1], train_X.shape[2]
n_features = 1 #da predirre
n_seq = 1
n_steps = 11*(n_in)

train_X = train_X.reshape((train_X.shape[0], n_seq, n_steps, n_features))
val_X = val_X.reshape((val_X.shape[0], n_seq, n_steps, n_features))
test_X = test_X.reshape((test_X.shape[0], n_seq, n_steps, n_features))

#Define Model
model = bi_lstm_model()
print(model.summary())

In [ ]:
history = model.fit(train_X, train_y, epochs=100, batch_size=30, validation_data=(val_X, val_y), verbose=1)

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
# make a prediction
yhat = model.predict(test_X)
print(test_X.shape)

In [ ]:
pred_denorm = [t*std+mean for t in yhat]
pred_df = pd.DataFrame(pred_denorm)
pred_df.head()

In [ ]:
test_denorm = [t*std+mean for t in test_y]
test = pd.DataFrame(test_denorm)
test.head()

In [ ]:
for i in range(n_out):
    
    plt.figure(figsize=(20,20))
#plot_test.reset_index(inplace=True,drop=True)
    plt.plot(test[1][5025:7000],label='x_test_'+str((i+1)*10))
    plt.plot(pred_df[1][5025:7000],label='predicted_'+str((i+1)*10))
    plt.legend()
    plt.title("DistrictCooling Facility Prediction at min: "+ str((i+1)*10))
    plt.savefig('min'+ str((i+1)*10)+'.png')
#print(plot_test[4:])
#print(pred_df[0:8])
#plot_test.reset_index(inplace=True,drop=True)
#print(plot_t.shape)
#print(pred_df.shape)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true+1))) * 100

In [ ]:
err=np.abs(pred_df-test)/(test+1)
err={}
for i in range(n_out):
    err[i]=[]
for i in range(len(test)):
    for r in range(n_out):
        if test[r][i]!=0:
            err[r].append(np.abs(pred_df[r][i]-test[r][i])/(test[r][i]))

In [ ]:
#Plot mean absolute percentage errore
mean=[]
for k in err.keys():
    mean.append(np.mean(err[k]))
plt.plot([v *100 for v in mean])
plt.title("MAPE for different horizon of prediction")
plt.xlabel("Distance")
plt.savwfig('MAPEhorizon.png')
plt.show()

In [ ]:

[m*100 for m in mean]

In [ ]:
def calculate_metrics(inv_y,inv_yhat):
	# calculate metrics
	rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
	mse = mean_squared_error(inv_y, inv_yhat)
	mae = mean_absolute_error(inv_y, inv_yhat)
	mape = mean_absolute_percentage_error(inv_y, inv_yhat)
	return rmse, mse, mae, mape

In [ ]:
rmse, mse, mae, mape = calculate_metrics(test, pred_df)

print("Test MSE: %.3f"%mse)
print('Test RMSE: %.3f' % rmse)
print("Test MAE: %.3f" % mae)
#print("Test MAPE: %.3f" % mape)

In [ ]:
## 'neurons': 12

# Total window size: 8 -> Input indices: [0 1 2 3 4 5] Label indices: [6 7]
# loss: 0.0151 - mean_absolute_percentage_error: 36.4191 - rmse: 89737.6441121866

# Total window size: 12 -> Input indices: [0 1 2 3 4 5] Label indices: [ 6  7  8  9 10 11]
# loss: 0.0378 - mean_absolute_percentage_error: 75.3893 - rmse: 165518.73420347186

# Total window size: 16 -> Input indices: [0 1 2 3 4 5 6 7 8 9] Label indices: [10 11 12 13 14 15]
# loss: 0.0358 - mean_absolute_percentage_error: 58.4344 - rmse: 168050.95017360165

# Total window size: 16 -> Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11] Label indices: [12 13 14 15]
# loss: 0.0278 - mean_absolute_percentage_error: 71.6392 - rmse: 140321.72971796765

# Total window size: 18 -> Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11] Label indices: [12 13 14 15 16 17]
# loss: 0.0356 - mean_absolute_percentage_error: 84.7530 - rmse: 176186.70476650228


## 

# Total window size: 216 (144 inputs, 72 outputs)
# 


